# How to use Google calendar API?

How to connect to Google calendar API?


In [ ]:
var execSync = require('child_process').execSync;
try {
    require.resolve('googleapis');
    require.resolve('google-auth-library');
} catch (e) {
    execSync('npm install googleapis google-auth-library');
}
var google = require('googleapis');
var googleAuth = require('google-auth-library');

var path = require('path');
var corePath = path.resolve(path.join(__dirname, '..', 'Core'));
var importer = require(corePath);
var seleniumPath = path.join('..', 'Selenium', 'How to authorize Google apps.ipynb');


How to list events?

In [ ]:

/**
 * Lists the next 10 events on the user's primary calendar.
 *
 * @param {google.auth.OAuth2} auth An authorized OAuth2 client.
 */
var resultEvents = [];
var listEvents = (auth, options = {calendarId: 'primary'}) => {
    if(typeof options['calendarId'] == 'undefined') {
        options['calendarId'] = 'primary';
    }
    var calendar = google.calendar('v3');
    var data = Object.assign({}, options, {
        auth: auth,
        //calendarId: 'p4685f6at8ccctenocjlqgnqss@group.calendar.google.com',
        //calendarId: 'rqqhq3me7ggg47chafhmeo6m3s@group.calendar.google.com',
        //q: 'work on SOS',
        maxResults: 100,
        singleEvents: true,
        orderBy: 'startTime'
    });
    return new Promise((resolve, reject) => {
        calendar.events.list(data, (err, response) => {
            try {
                var events = processResult(err, response);
                resolve(events);
            } catch (e) {
                reject(e);
            }
        });
    });
}

(listEvents);


In [ ]:

var listEventBetweenTwoDates = (auth, min, max) => {
    return listEvents(auth, {
        timeMin: ISODateString(min),
        timeMax: ISODateString(max),
    });
};


In [ ]:
var processResult = (err, response) => {
    if (err) {
      console.log('The API returned an error: ' + err);
      throw err;
    }
    resultEvents = []; // reset the array
    var events = response.items;
    if (events.length == 0) {
        console.log('No events found.');
    } else {
        for (var i = 0; i < events.length; i++) {
            var event = events[i];
            var start = event.start.dateTime || event.start.date;
            resultEvents[resultEvents.length] = {
                start: new Date(start), event: event};
            console.log('%s - %s', start, event.summary);
        }
    }
    return resultEvents;
}


How to test Google calendar API?

In [ ]:

// Authorize a client with the loaded credentials, then call the
  // Google Calendar API.
$$.async();
var authorize, oauth2Client;
importer.importNotebook(seleniumPath)
    .then(r => authorize = r['authorize'])
    .then(() => authorize(['https://www.googleapis.com/auth/calendar']))
    .then(c => {
        var auth = new googleAuth();
        oauth2Client = new auth.OAuth2(
            c.clientId_, c.clientSecret_, c.redirectUri_);
        oauth2Client.credentials = c.credentials;
        return oauth2Client;
    })
    .then(r => listEvents(r))
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e));
